# 클러스터 구성 과정상
# 1. 소규모로 클러스터 생성
# 2. 설정
# 3. 노드 다시 켜기
## 의 과정을 거쳐야 한다. (aws는 k8s에 최적화가 덜되서...)

jupyter가 싫으면 .py로 해도 상관 없을듯 아마?

절차

1. node_size.yml => 1
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. aws eks --region us-east-1 update-kubeconfig --name test-eks-cluster
4. kubectl create -f ./k8s-manifests/eni-configs/.
5. kubectl set env daemonset aws-node -n kube-system AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG=true
6. kubectl set env daemonset aws-node -n kube-system ENI_CONFIG_LABEL_DEF=failure-domain.beta.kubernetes.io/zone
7. node_size.yml => 0
8. terraform apply
9. node_size.yml => cluster size you want
10. terraform apply

# 1. 초기 클러스터 크기

In [4]:
import yaml
def change_node_size_yml(desired_size, max_size, min_size):
    with open('node_size.yml') as f:
        node_size = yaml.load(f, Loader=yaml.FullLoader)
        print('current_state',node_size)
        
    node_size['desired_size'] = desired_size
    node_size['min_size'] = max_size
    node_size['max_size'] = min_size
    
    print('after change', node_size)
    
    with open('node_size.yml', 'w') as f:
        yaml.dump(node_size,f)

In [5]:
desired_size = 0
max_size = 0
min_size = 1

In [6]:
change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


# 2. Terraform 실행
=> 여기서만 딱 안되네... 일단 테라폼 은 따로 돌리고 로직 자체는 되나 한번 보자

In [3]:
#!terraform init

In [1]:
#!terraform plan

In [2]:
#여기서 키 얻는데 실패해서 자꾸 dead lock 걸림
#!terraform apply --auto-approve

In [ ]:
#!terraform destroy --auto-approve

# 3. k8s 마스터 - pc 연결
=> aws cli도 안되네?;;;

In [ ]:
#!aws eks --region us-east-1 update-kubeconfig --name test-eks-cluster

# 4,5,6 기본 셋팅

In [ ]:

kubectl create -f ./k8s-manifests/eni-configs/.
kubectl set env daemonset aws-node -n kube-system AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG=true
kubectl set env daemonset aws-node -n kube-system ENI_CONFIG_LABEL_DEF=failure-domain.beta.kubernetes.io/zone


# 7,8 node size -> 0

In [14]:
#node_size.yml => 0

In [16]:
desired_size = 0
max_size = 0
min_size = 1 #max는 최소 1은 되어야 함.

change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 0, 'max_size': 0, 'min_size': 0}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


In [12]:
#!terraform apply --auto-approve

In [ ]:
#인스턴스 종료 들어갈때 까지 시간이 좀 들어감
#terraform은 instance에 직접 명령내리는게 아니라 - node group한테 명령한거고
#node group - autoscaling group - instance 순서를 거쳐서 종료되기 떄문에
import time

time.sleep(120)

# 9,10 node size -> 다시 늘리기

In [ ]:
#node_size.yml => cluster size you want

In [7]:
desired_size = 1
max_size = 1
min_size = 1

change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 1, 'max_size': 1, 'min_size': 1}


In [13]:
#!terraform apply --auto-approve